# Entraînement YOLO pour la détection de bulles de texte

Ce notebook permet d'entraîner un modèle YOLO pour détecter les bulles de texte dans les images de manga.


In [1]:
# Installation des dépendances
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [2]:
# Import des bibliothèques nécessaires
from ultralytics import YOLO
import os


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Configuration du dataset

Nous allons utiliser le dataset déjà préparé dans le dossier `data/train_version_YOLO_V8`


In [3]:
# D'abord je décompresse  mon dossier zip
!unzip Scan-Trad-project.zip

Archive:  Scan-Trad-project.zip
   creating: Scan-Trad-project/
   creating: Scan-Trad-project/.git/
  inflating: Scan-Trad-project/.git/config  
  inflating: Scan-Trad-project/.git/description  
  inflating: Scan-Trad-project/.git/FETCH_HEAD  
  inflating: Scan-Trad-project/.git/HEAD  
   creating: Scan-Trad-project/.git/hooks/
  inflating: Scan-Trad-project/.git/hooks/applypatch-msg.sample  
  inflating: Scan-Trad-project/.git/hooks/commit-msg.sample  
  inflating: Scan-Trad-project/.git/hooks/fsmonitor-watchman.sample  
  inflating: Scan-Trad-project/.git/hooks/post-update.sample  
  inflating: Scan-Trad-project/.git/hooks/pre-applypatch.sample  
  inflating: Scan-Trad-project/.git/hooks/pre-commit.sample  
  inflating: Scan-Trad-project/.git/hooks/pre-merge-commit.sample  
  inflating: Scan-Trad-project/.git/hooks/pre-push.sample  
  inflating: Scan-Trad-project/.git/hooks/pre-rebase.sample  
  inflating: Scan-Trad-project/.git/hooks/pre-receive.sample  
  inflating: Scan-Trad-proj

## Séparer mes données en train/val

In [5]:
import os
import shutil
import random

# Dossiers d'entrée
img_dir = '/content/Scan-Trad-project/data/train_version_YOLO_V8/images'
lbl_dir = '/content/Scan-Trad-project/data/train_version_YOLO_V8/labels'

# Dossiers de sortie
for subset in ['train', 'val']:
    os.makedirs(f'/content/Scan-Trad-project/data/train_version_YOLO_V8/{subset}/images', exist_ok=True)
    os.makedirs(f'/content/Scan-Trad-project/data/train_version_YOLO_V8/{subset}/labels', exist_ok=True)

# Séparation (80% train, 20% val)
images = [f for f in os.listdir(img_dir) if f.endswith('.jpg')]
random.shuffle(images)
split = int(0.8 * len(images))
train_imgs = images[:split]
val_imgs = images[split:]

def move_files(file_list, subset):
    for img_file in file_list:
        lbl_file = img_file.replace('.jpg', '.txt')
        shutil.copy(os.path.join(img_dir, img_file), f'/content/Scan-Trad-project/data/train_version_YOLO_V8/{subset}/images/{img_file}')
        shutil.copy(os.path.join(lbl_dir, lbl_file), f'/content/Scan-Trad-project/data/train_version_YOLO_V8/{subset}/labels/{lbl_file}')

move_files(train_imgs, 'train')
move_files(val_imgs, 'val')


## Ecrire dans un fichier json

In [6]:
%%writefile data.yaml
path: /content/Scan-Trad-project/data/train_version_YOLO_V8  # Chemin racine
train: train/images
val: val/images

nc: 1  # nombre de classes
names: ['bulle']  # nom de ta classe


Writing data.yaml


## Entraînement du modèle

Nous allons utiliser YOLOv8n comme modèle de base et faire du transfer learning


In [7]:
# Chargement du modèle YOLOv8n
model = YOLO('yolov8n.pt')

# Entraînement du modèle
results = model.train(
    data='data.yaml',   # config data
    epochs=100,         # nombre d'époques
    imgsz=640,          # taille des images
    batch=16,           # taille du batch
    patience=20,        # early stopping
    save=True,          # sauvegarder le meilleur modèle
    device='0'          # utiliser le GPU
)


100%|██████████| 6.25M/6.25M [00:00<00:00, 106MB/s]


Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, 

100%|██████████| 755k/755k [00:00<00:00, 24.3MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


...

    100/100      4.27G     0.8266     0.4821      0.944        106        640: 100%|██████████| 6/6 [00:01<00:00,  4.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.06it/s]

                   all         39        259      0.979      0.992      0.994      0.801



100 epochs completed in 0.060 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.16it/s]


                   all         39        259      0.979      0.992      0.994      0.801
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to runs/detect/train


## Évaluation du modèle


In [8]:
# Évaluation sur le jeu de validation
metrics = model.val()
print(f"mAP50: {metrics.box.map50}")
print(f"mAP50-95: {metrics.box.map}")


Ultralytics 8.3.149 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1137.8±372.7 MB/s, size: 97.8 KB)


val: Scanning /content/Scan-Trad-project/data/train_version_YOLO_V8/val/labels.cache... 39 images, 0 backgrounds, 0 corrupt: 100%|██████████| 39/39 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.85it/s]


                   all         39        259      0.979      0.992      0.994      0.801
Speed: 5.0ms preprocess, 12.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train2
mAP50: 0.9943759531489729
mAP50-95: 0.8012605348787212


## Test du modèle sur une image


In [11]:
# Prédiction sur une image
results = model.predict('/content/Scan-Trad-project/data/train_version_YOLO_V8/images/DS_100_jpg.rf.f4aa7b39cf6ec62c4b8de2fd066c3a4b.jpg', save=True)
print("Prédictions terminées !")



image 1/1 /content/Scan-Trad-project/data/train_version_YOLO_V8/images/DS_100_jpg.rf.f4aa7b39cf6ec62c4b8de2fd066c3a4b.jpg: 640x640 7 bulles, 8.5ms
Speed: 2.7ms preprocess, 8.5ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/train3
Prédictions terminées !


## Sauvegarder le modèle


In [12]:
model.save('best_textbubbles_model.pt')
